In [2]:
import numpy as np
import sklearn.preprocessing as prep
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
def xavier_init(fan_in,fan_out,constant=1):
    low = -constant*np.sqrt(6.0/(fan_in+fan_out))
    high = constant*np.sqrt(6.0/(fan_in+fan_out))    
    return tf.random_uniform((fan_in,fan_out),minval = low,maxval = high,dtype = tf.float32)

class AdditiveGaussianNoiseAutoencoder(object):
    def __init__(self,n_input,n_hidden,transfer_function=tf.nn.softplus,
                optimizer = tf.train.AdamOptimizer(),scale=0.1):
        self.n_input = n_input
        self.n_hidden = n_hidden
        self.transfer = transfer_function
        self.scale = tf.placeholder(tf.float32)
        self.training_scale = scale
        network_weights = self._initialize_weights()
        self.weights = network_weights      
        self.x = tf.placeholder(tf.float32,[None,self.n_input])
        self.hidden = self.transfer(tf.add(tf.matmul(
                                    self.x + scale*tf.random_normal((n_input,)),
                                    self.weights['w1']),self.weights['b1']))
        self.reconstruction = tf.add(tf.matmul(self.hidden,
                                              self.weights['w2']),self.weights['b2'])
        self.cost = 0.5*tf.reduce_sum(tf.pow(tf.subtract(
                                        self.reconstruction,self.x),2.0))
        self.optimizer = optimizer.minimize(self.cost)
        init = tf.global_variables_initializer()
        self.sess = tf.Session()
        self.sess.run(init)
    def _initialize_weights(self):
        all_weights = dict()
        all_weights['w1'] = tf.Variable(xavier_init(self.n_input,
                                                    self.n_hidden))
        all_weights['b1'] = tf.Variable(tf.zeros([self.n_hidden],
                                                 dtype = tf.float32))
        all_weights['w2'] = tf.Variable(tf.zeros([self.n_hidden,
                                                    self.n_input],dtype=tf.float32))
        all_weights['b2'] = tf.Variable(tf.zeros([self.n_input],
                                                 dtype = tf.float32))
        return all_weights
    
    def partial_fit(self,X):
        cost,opt = self.sess.run((self.cost,self.optimizer),
                                feed_dict = {self.x:X,self.scale:self.training_scale})
        return cost 
    def calc_total_cost(self,X):
        return self.sess.run(self.cost,feed_dict={self.x:X,
                                                 self.scale:self.training_scale
                                                 })
    def transform(self,X):
        return self.sess.run(self.hidden,feed_dict={self.x:X,
                                                    self.scale:self.training_scale
                                                   })
    def generate(self,hidden=None):
        if hidden is None:
            hidden = np.random.normal(size = self.weights["b1"])
        return self.sess.run(self.reconstruction,
                            feed_dict={self.hidden:hidden})
    def reconstruction(self,X):
        return self.sess.run(self.reconstruction,feed_dict={self.x:X,
                            self.scale:self.training_scale
                            })
    def getWeights(self):
        return self.sess.run(self.weights["w1"])
    def getBiases(self):
        return self.sess.run(self.weights["b1"])

def standard_scale(X_train,X_test):
    preprocessor = prep.StandardScaler().fit(X_train)
    X_train = preprocessor.transform(X_train)
    X_test = preprocessor.transform(X_test)    
    return X_train,X_test

def get_random_block_from_data(data,batch_size):
    start_index = np.random.randint(0,len(data)-batch_size)
    return data[start_index:(start_index+batch_size)]
    
mnist = input_data.read_data_sets("/home/yanbin/data/MNIST_data/",one_hot=True)
X_train,X_test = standard_scale(mnist.train.images,mnist.test.images)
n_samples = int(mnist.train.num_examples)
training_epochs = 20
batch_size = 128
display_step =1
autoencoder = AdditiveGaussianNoiseAutoencoder(n_input=784,
                n_hidden = 200,
                transfer_function = tf.nn.softplus,
                optimizer = tf.train.AdamOptimizer(learning_rate=0.001),
                scale = 0.01)

for epoch in range(training_epochs):
    avg_cost =0 
    total_batch = int(n_samples/batch_size)
    for i in range(total_batch):
        batch_xs = get_random_block_from_data(X_train,batch_size)
        cost = autoencoder.partial_fit(batch_xs)
        avg_cost = cost / n_samples*batch_size
        
    if epoch % display_step ==0:
        print "Epoch:",'%04d'% (epoch+1), "cost=","{:.9f}".format(avg_cost)

Extracting /home/yanbin/data/MNIST_data/train-images-idx3-ubyte.gz
Extracting /home/yanbin/data/MNIST_data/train-labels-idx1-ubyte.gz
Extracting /home/yanbin/data/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting /home/yanbin/data/MNIST_data/t10k-labels-idx1-ubyte.gz
Epoch: 0001 cost= 15.105495455
Epoch: 0002 cost= 18.599398864
Epoch: 0003 cost= 14.589381818
Epoch: 0004 cost= 11.500264773
Epoch: 0005 cost= 16.021156818
Epoch: 0006 cost= 9.939811364
Epoch: 0007 cost= 18.354043182
Epoch: 0008 cost= 27.202600000
Epoch: 0009 cost= 76.856600000
Epoch: 0010 cost= 13.460350000
Epoch: 0011 cost= 12.290792045
Epoch: 0012 cost= 11.072611364
Epoch: 0013 cost= 10.515194318
Epoch: 0014 cost= 14.870626136
Epoch: 0015 cost= 14.216250000
Epoch: 0016 cost= 13.709606818
Epoch: 0017 cost= 15.534497727
Epoch: 0018 cost= 10.117022727
Epoch: 0019 cost= 9.190336364
Epoch: 0020 cost= 47.862604545


In [3]:
print "Total cost: " + str(autoencoder.calc_total_cost(X_test))


Total cost: 642356.0
